In [1]:
import pandas as pd 
import numpy as np 
import pathlib as path
import matplotlib.pyplot as plt
import random
from PIL import Image
from random import sample
from cmath import nan
import json

In [2]:
#define the relevant path to the current batch you wish to parse
gap_name='pilot_one_week' #dont change this as this parsing sciprt only handles the same day batches.  

project_name='birds'
experiment_name='experiment 1'
batch_name='batch 2'

PATH_TO_BATCH=path.Path.cwd().parent.parent.parent / 'projects' / project_name / experiment_name / 'collected data' / gap_name / batch_name
PATH_TO_BATCH_DATA = PATH_TO_BATCH / 'data'

if PATH_TO_BATCH.exists():
    print('batch folder found at:\n' ,PATH_TO_BATCH)
else: 
    print('path to batch is non existent:\n',PATH_TO_BATCH)

batch folder found at:
 c:\Users\User\Desktop\visual_memory_project\visual-memory-project\projects\birds\experiment 1\collected data\pilot_one_week\batch 2


In [3]:


qualification_name_for_testin='UPDATE-eligible for testing 1 week birds' #note that the -space- after the title is improtant as for some reason this is how the qualification name is defined
qualification_name_for_entire_experiment='UPDATE-completed birds memory rep'



In [4]:
#this section loads the shelf dict in the state it is after finishing the testing session for this batch. 
shelf_dict_after_test_name=PATH_TO_BATCH / 'shelf after encoding session closed.txt' #define the name of the relevant shelf for this stage
with open(shelf_dict_after_test_name) as f:
    data = f.read()
shelf_dict = json.loads(data)
shelf_dict

{'A21UA6O7ZFAIQJ': [33, 1672618541424, 1673265610931],
 'ABVBYS63BZWZ6': [54, 1672691441654, 1673184321835, 1673268961032],
 'AN4P0SU1YSPF2': [30,
  1672608091759,
  1673108004931,
  1673115273107,
  1673136507681,
  1673150501613,
  1673188398470],
 'AOOLS8280CL0Z': [7,
  1672601000146,
  1673111749669,
  1673137735525,
  1673185641801],
 'A2I6ZALE49CVSC': [0, 1673367367444],
 'A1TPGXT718D68E': [1, 1673367560992],
 'A2MOU3YD664C0H': [2, 1673367619383],
 'A2LA3NEWBU30QT': [3, 1673367650470],
 'A3DP9IH8W5OXJL': [4, 1673367667732],
 'AMAEYJH3DWA1V': [5, 1673367724382],
 'AYCD7UMZHW4WD': [6, 1673367821150],
 'ATH176GTV3ABP': [8, 1673368064110],
 'A1M1W56A3UR5PE': [9, 1673368158334],
 'A3S3WYVCVWW8IZ': [10, 1673368196632],
 'A21ZYNCWJLHMCY': [999, 999, 999],
 'A1SJLXB0XFCJ97': [999, 999, 999],
 'A2RSBH1E3TEESG': [11, 1673368592968],
 'A3QO3HIV92H659': [12, 1673368687949],
 'A1SJLXB0XFCJ92': [13, 1673368956444],
 'A2T5CROJ0FWIJY': [14, 1673369034005],
 'AV55CMG8KR1YH': [999, 999, 999],
 'A2

In [6]:
#this section reads all the relevant files: 
if (PATH_TO_BATCH / 'Batch_encoding_batch_results.csv').exists():
    encoding_df=pd.read_csv(PATH_TO_BATCH / 'Batch_encoding_batch_results.csv')
else: 
    print(f'no "Batch_encoding_batch_results.csv" was found in data')

if (PATH_TO_BATCH / 'Batch_testing_batch_results.csv').exists():
    testing_df=pd.read_csv(PATH_TO_BATCH / 'Batch_testing_batch_results.csv')
else: 
    print(f'no "Batch_testing_batch_results.csv" was found in data (probably because only the encoding session finished)')
if (PATH_TO_BATCH / 'Batch_workers.csv').exists():
    workers_df=pd.read_csv(PATH_TO_BATCH / 'Batch_workers.csv')
else:
    print('No AMAZON worker list was found: please make sure to upload one and name it correctly')



no "Batch_testing_batch_results.csv" was found in data (probably because only the encoding session finished)


In [7]:
def find_largest_consequtive_repetition(responses_array):
#add a stop value so the loop will use the logic also if the end of the responses is all repeating: 
    responses_array=np.append(responses_array,[99])
    longest_rep_dict=dict()
    last_input=responses_array[0]
    consequtive_rep_counter=0
    for response in responses_array[1:]: 
        if response==last_input: #if consequtive rep: 
            consequtive_rep_counter=consequtive_rep_counter+1 #add to counter
        else: #once a new entry is in the vector - store the counter rep information 
            if last_input in longest_rep_dict.keys(): #if it was allready stored in our dictionary
                if longest_rep_dict[last_input]<consequtive_rep_counter: #replace the counter only if it is higher than what stored in dict
                    longest_rep_dict[last_input]=consequtive_rep_counter
            else: #this entry is not yet in the dictionary: add it 
                longest_rep_dict[last_input]=consequtive_rep_counter
            
            consequtive_rep_counter=0
            last_input=response
    return max(longest_rep_dict.values())

In [8]:
def process_worker_results(PATH_TO_BATCH_DATA,subject_name):
    cur_sub=pd.read_csv(PATH_TO_BATCH_DATA / subject_name)
    sub_demographics=cur_sub[['workID','Age','Gender']].iloc[0]


    #extract demo related information: 

    #if participant did not click on the arrow during the demo a 'demo_encoding_response.rt' colmumn wont exists: 
    #thus we will create one and fill it with nans (so everything will be consistent with other participants)
    if not('demo_encoding_response.rt' in cur_sub.columns):
        cur_sub['demo_encoding_response.rt']=np.nan
    demo_columns=['demo_encoding_loop.thisTrialN','DemoImage','DemoCorrect','demo_encoding_response.rt','demo_encoding_response.keys']

    sub_demo_information=cur_sub[demo_columns]
    empty_inds=sub_demo_information.loc[sub_demo_information.isnull().apply(lambda x: all(x), axis=1)].index
    #drop irrelevant columns: 
    sub_demo_information=sub_demo_information.drop(index=empty_inds)


    #extract the demo test columns: 
    demo_test_columns=['demo_test_response.keys','demo_test_response.corr','demo_test_response.rt','demo_test_loop.thisTrialN','DemoImage1','DemoImage2','DemoCorrectTest']
    sub_demo_test_information=cur_sub[demo_test_columns]
    empty_inds=sub_demo_test_information.loc[sub_demo_test_information.isnull().apply(lambda x: all(x), axis=1)].index
    #drop irrelevant columns: 
    sub_demo_test_information=sub_demo_test_information.drop(index=empty_inds)

    demo_df=pd.concat([sub_demo_information.reset_index(),sub_demo_test_information.reset_index()],axis=1)

    #extract real experiment related information: 

    #if participant did not click on the arrow during the encoding a 'test_encoding_response.rt' column wont exists: 
    #thus we will create one and fill it with nans (so everything will be consistent with other participants)
    if not('test_encoding_response.rt' in cur_sub.columns):
        cur_sub['test_encoding_response.rt']=np.nan


    encoding_related_columns=['test_encoding_response.keys','test_encoding_response.corr','trials.thisTrialN','target_image','pair','layer','correct','test_encoding_response.rt','key_resp_end.keys']
    sub_encoding_information=cur_sub[encoding_related_columns]
    #encoding section ends with a key press of the space key: so seperate this phase by finding this space key row
    end_of_section_ind=np.where(sub_encoding_information['key_resp_end.keys']=='space')[0][0]
    sub_encoding_information=sub_encoding_information.iloc[0:end_of_section_ind]
    #remove all the rows that precede the real encoding phase: 
    empty_inds=sub_encoding_information.loc[sub_encoding_information.isnull().apply(lambda x: all(x), axis=1)].index
    sub_encoding_information=sub_encoding_information.drop(index=empty_inds).reset_index()

    #extract real experiment TEST related information: 
    #test_related_columns=['layer','correct','test_test_response.keys','test_test_response.corr','test_test_response.rt','trials_2.thisRepN','trials_2.thisTrialN','trials_2.thisN','trials_2.thisIndex','trials_2.ran','image1','image2']
    #sub_test_information=cur_sub[test_related_columns].iloc[end_of_section_ind+2:-1]


    subject_dictionary=dict()
    subject_dictionary['demographics']=sub_demographics
    subject_dictionary['demo_df']=demo_df
    subject_dictionary['encoding_df']=sub_encoding_information
    #subject_dictionary['test_df']=sub_test_information

    return subject_dictionary



In [9]:

#get all results with Encoding information - extract the arrow attention check accuracy and RT (RT is currently not usd as a criterion)

#this section extract the list of participants from the downloaded results files (and not via the workers or session list csvs) 
# - it will create the qualification_df (a table with information on the worker ids and encoding behavior of all participants that we have files for)
all_filenames=[file for file in PATH_TO_BATCH_DATA.iterdir() if 'csv' in file.name and 'ENCODING' in file.name]

#print(f'current csv files:\n{all_filenames}')

qualification_for_test_df=pd.DataFrame(columns=['workerID','arrow_acc','mean_arrow_RT'])
for subject_csv in all_filenames:



    #thojs is a patch for the case of saving non complete files (some are ecompletly eomptyu, and otherd dont have some important demo columns)
    if (PATH_TO_BATCH_DATA / path.Path(subject_csv)).stat().st_size <= 5:
        print(f' subject: {subject_csv} did not have a demo column and thus disqualified')
        continue

    cur_sub=pd.read_csv(PATH_TO_BATCH_DATA / path.Path(subject_csv))
    #get rid of participants with encoding files that are basiscly empty:
    if (not('demo_test_response.keys' in cur_sub.columns)):
        print(f' subject: {subject_csv} did not have a demo column and thus disqualified')
        continue
        


    subject_dict=process_worker_results(PATH_TO_BATCH_DATA,path.Path(subject_csv))
    cur_sub_encoding=subject_dict['encoding_df']

    sname=subject_csv.name.split('_')[1] #change according to actuall format. 

    #get only attention check related info: 
    cur_sub_encoding=cur_sub_encoding[['test_encoding_response.keys','correct','test_encoding_response.rt']].copy().dropna(how = 'all')
    if len(cur_sub_encoding)==0: #empty - the participant didnt respond on the arrow at all
        arrow_acc=0
        RT=nan
    else: 
        RT=cur_sub_encoding['test_encoding_response.rt'].mean()
        arrow_acc=(cur_sub_encoding['correct']==cur_sub_encoding['test_encoding_response.keys']).mean()
    sub_df=pd.DataFrame({'workerID':sname,'arrow_acc':arrow_acc,'mean_arrow_RT':RT},index=[sname])
    qualification_for_test_df=pd.concat([qualification_for_test_df,sub_df],axis=0)

#the following part update the qualification_df with information on wether the participant id exists in the amazon workers list: 

#change participants qualifications if they exists in the workers list based on thier encoding arrow accuracy
qualification_for_test_df['in_encoding_workers_list']=nan

for curr_worker_ID in qualification_for_test_df.index:
    if curr_worker_ID in encoding_df['WorkerId'].values:
        qualification_for_test_df.loc[curr_worker_ID,'in_encoding_workers_list']=int(1)
    else:
        qualification_for_test_df.loc[curr_worker_ID,'in_encoding_workers_list']=int(0)


## the section below seperate between 4 types of participants: 
    * qualified and in the workers list
    * qualified and not in the workers list 
    * disqualified and in the worker list
    * disqualified and not in the worker list

In [10]:
qualified_df=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']>=0.6) & (qualification_for_test_df['in_encoding_workers_list']==1),:].copy()
print('qualified participants')
qualified_df

qualified participants


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A1122EUMFXHF38,A1122EUMFXHF38,1.0,1.009800,1.0
A1EH9BPKYXFBS5,A1EH9BPKYXFBS5,1.0,0.991700,1.0
A1I5KX8TP3K5L2,A1I5KX8TP3K5L2,1.0,1.258740,1.0
A1OQFPHKZV0LM5,A1OQFPHKZV0LM5,0.8,0.955580,1.0
A1P2XKONWAOXHP,A1P2XKONWAOXHP,1.0,0.970920,1.0
A1T2JRYTUVUD2O,A1T2JRYTUVUD2O,1.0,0.907400,1.0
A1T6TTD7XCUHOU,A1T6TTD7XCUHOU,0.8,1.912750,1.0
A1TPGXT718D68E,A1TPGXT718D68E,1.0,1.272320,1.0
A1VBA5TN6G265Q,A1VBA5TN6G265Q,1.0,1.015880,1.0
A1Y07DW8W2UXSD,A1Y07DW8W2UXSD,1.0,1.250600,1.0


In [11]:
qualified_df_not_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']>=0.6) & (qualification_for_test_df['in_encoding_workers_list']==0),:].copy()
print('qualified participants that are not in the workers list (and thus we cant add a qualification for them)')
qualified_df_not_in_workers_list

qualified participants that are not in the workers list (and thus we cant add a qualification for them)


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list


In [12]:
disqualified_df_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']<0.6) & (qualification_for_test_df['in_encoding_workers_list']==1),:].copy()
print('disqualified participants that are in the workers list:')
disqualified_df_in_workers_list

disqualified participants that are in the workers list:


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A208AK4JNI6PZ0,A208AK4JNI6PZ0,0.4,1.8017,1.0
A2OK9P9QCBZKMB,A2OK9P9QCBZKMB,0.0,NaN,1.0
A2YK6945O0SP8N,A2YK6945O0SP8N,0.0,NaN,1.0
AAZNLKG9ZCL1N,AAZNLKG9ZCL1N,0.0,NaN,1.0
ATP0UFAVIANFL,ATP0UFAVIANFL,0.2,0.1645,1.0
AYCD7UMZHW4WD,AYCD7UMZHW4WD,0.0,NaN,1.0


In [13]:
disqualified_df_not_in_workers_list=qualification_for_test_df.loc[(qualification_for_test_df['arrow_acc']<0.6) & (qualification_for_test_df['in_encoding_workers_list']==0),:].copy()
print('disqualified participants that are NOT in the workers list:')
disqualified_df_not_in_workers_list

disqualified participants that are NOT in the workers list:


,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list


In [14]:
qualified_df['type']='qualified in workers list'
qualified_df_not_in_workers_list['type']='qualified not in workers list'
disqualified_df_in_workers_list['type']='disqualified in workers list'
disqualified_df_not_in_workers_list['type']='disqualified not in workers list'
info_from_all_downloaded_datafiles=pd.concat([qualified_df,qualified_df_not_in_workers_list,disqualified_df_in_workers_list,disqualified_df_not_in_workers_list],axis=0)
info_summary_df=info_from_all_downloaded_datafiles.groupby('type').aggregate({'arrow_acc':'count'})
info_summary_df['worker_ids']=nan
info_summary_df['worker_ids']=info_summary_df['worker_ids'].astype(object)
if 'qualified in workers list' in info_summary_df.index: 
    info_summary_df.at['qualified in workers list','worker_ids']=qualified_df.index.to_list()
if 'qualified not in workers list' in info_summary_df.index:    
    info_summary_df.at['qualified not in workers list','worker_ids']=qualified_df_not_in_workers_list.index.to_list()
if 'disqualified in workers list' in info_summary_df.index: 
    info_summary_df.at['disqualified in workers list','worker_ids']=disqualified_df_in_workers_list.index.to_list()
if 'disqualified not in workers list' in info_summary_df.index: 
    info_summary_df.at['disqualified not in workers list','worker_ids']=disqualified_df_not_in_workers_list.index.to_list()

info_summary_df

,arrow_acc,worker_ids
type,,
disqualified in workers list,6,"[A208AK4JNI6PZ0, A2OK9P9QCBZKMB, A2YK6945O0SP8..."
qualified in workers list,41,"[A1122EUMFXHF38, A1EH9BPKYXFBS5, A1I5KX8TP3K5L..."


In [15]:
#update the workers list with the relevant qualification: 
new_shelf_dict=shelf_dict.copy()

#add the qualification for all eligible participants for the test session: 
ind_in_workers_df=[]
print('\nupdating qualifications for participants elligable for test session\n',list(qualified_df.index))
for worker_id in qualified_df.index:
    if worker_id in workers_df['Worker ID'].values:
        workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_testin]=1
        ind_in_workers_df.append(workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_testin].index[0])

print('\nupdating qualifications for participants elligable for test session that are not in the workers list\n',list(qualified_df_not_in_workers_list.index))
for worker_id in qualified_df_not_in_workers_list.index: 
    if worker_id in shelf_dict.keys():
        prev_entriees=shelf_dict[worker_id]
        cur_entriees=prev_entriees
        cur_entriees[0]=999
        new_shelf_dict[worker_id]=cur_entriees
        print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#remove from experiment all those that disqualified (so they wont be able to take the test session): 
print('\nupdating qualifications for participants that were disqualified for test session and are in the workers list\n',list(disqualified_df_in_workers_list.index))
for worker_id in disqualified_df_in_workers_list.index: 
    if worker_id in workers_df['Worker ID'].values:
        workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_entire_experiment]=1
        ind_in_workers_df.append(workers_df.loc[workers_df['Worker ID']==worker_id,qualification_name_for_entire_experiment].index[0])
        
        if worker_id in shelf_dict.keys(): #change thier shelf entree (so they wont waste a valid csv number)
            prev_entriees=shelf_dict[worker_id]
            cur_entriees=prev_entriees.copy()
            cur_entriees[0]=999
            new_shelf_dict[worker_id]=cur_entriees
            print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#handle those that disqualified but are not in the amazon worker list: (change thier shelf entries to 999)
print('\nhandling participants that exists in the shelf but not in the amazon workers list (reseting thier shelf entriees)\n',list(disqualified_df_not_in_workers_list.index))
for worker_id in disqualified_df_not_in_workers_list.index: 
    if worker_id in shelf_dict.keys():
        prev_entriees=shelf_dict[worker_id]
        cur_entriees=prev_entriees.copy()
        cur_entriees[0]=999
        new_shelf_dict[worker_id]=cur_entriees
        print(f'changed subject {worker_id} previous shelf entriee: {prev_entriees} to {cur_entriees} in the shelf')


#print the two relevant columns of the updated workers_list: 
print('these are all the participants that will be updated in the workers list:')
workers_df.loc[ind_in_workers_df,['Worker ID',qualification_name_for_testin,qualification_name_for_entire_experiment]]



updating qualifications for participants elligable for test session
 ['A1122EUMFXHF38', 'A1EH9BPKYXFBS5', 'A1I5KX8TP3K5L2', 'A1OQFPHKZV0LM5', 'A1P2XKONWAOXHP', 'A1T2JRYTUVUD2O', 'A1T6TTD7XCUHOU', 'A1TPGXT718D68E', 'A1VBA5TN6G265Q', 'A1Y07DW8W2UXSD', 'A1YC558J4E5KZ', 'A249ZABI42JTSS', 'A28P7V6WDT9UN1', 'A290Z6QAL17PQE', 'A2I6ZALE49CVSC', 'A2KLIBLDVDV144', 'A2L18Q7871EONK', 'A2LA3NEWBU30QT', 'A2MOU3YD664C0H', 'A2OT9ZJC4MQBS8', 'A2PAWE1NW4MD8L', 'A2RSBH1E3TEESG', 'A2RWLFIP23FZA7', 'A2T5CROJ0FWIJY', 'A30RAYNDOWQ61S', 'A34LSCB90T5KYE', 'A36BPBYVFDSNBB', 'A3D3FNUSK1A2P6', 'A3DP9IH8W5OXJL', 'A3EKR5FNVVN2CC', 'A3G55RJTW3BSGM', 'A3NQWRK8KWTILX', 'A3QO3HIV92H659', 'A3S3WYVCVWW8IZ', 'A66QAY9ZC7PS4', 'A9500TJQDB94E', 'AMAEYJH3DWA1V', 'AQN32EZB7OQRG', 'AT5LT9B0GKOIK', 'ATH176GTV3ABP', 'AVSPYLB9KOHMX']

updating qualifications for participants elligable for test session that are not in the workers list
 []

updating qualifications for participants that were disqualified for test session and are in 

,Worker ID,UPDATE-eligible for testing 1 week birds,UPDATE-completed birds memory rep
21,A1122EUMFXHF38,1.0,NaN
255,A1EH9BPKYXFBS5,1.0,NaN
320,A1I5KX8TP3K5L2,1.0,NaN
436,A1OQFPHKZV0LM5,1.0,NaN
445,A1P2XKONWAOXHP,1.0,NaN
533,A1T2JRYTUVUD2O,1.0,NaN
536,A1T6TTD7XCUHOU,1.0,NaN
548,A1TPGXT718D68E,1.0,NaN
582,A1VBA5TN6G265Q,1.0,NaN
631,A1Y07DW8W2UXSD,1.0,NaN


In [16]:
#save the worker list to upload (to update the qualifications) -> this is what you upload to amazon to update the qualifications of relevant participants
workers_df.to_csv(PATH_TO_BATCH/'Batch_workers_for_upload_after_encoding.csv',index=False)

c:\Users\User\Desktop\visual_memory_project\visual-memory-project\.venv\lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)


In [17]:
#this section remove participants that were assigned with an unavailble csv from the shelf (also not changing qualifications for them at all so they could come back to another encoding session in the future) 
max_csv_number=62
participants_exceeding_available_csvs=[]
for key in shelf_dict.keys():
    cur_values=shelf_dict[key]
    if (cur_values[0]>max_csv_number) and (not(cur_values[0]>=998)):
        participants_exceeding_available_csvs.append(key)
        if key in new_shelf_dict.keys():
            del new_shelf_dict[key]

print(f'the following participants were removed from the shelf dictionary due to exceeding the maximum available encoding csv file of {max_csv_number}:\nthis probably means that they did not even see the encoding of the experiment and should be considered as never participated in our\n{participants_exceeding_available_csvs}')

the following participants were removed from the shelf dictionary due to exceeding the maximum available encoding csv file of 62:
this probably means that they did not even see the encoding of the experiment and should be considered as never participated in our
[]


In [18]:
#this section gives 999 to participants that are in the shelf but for some reason are not in the batch_results_file (so they probably saw the experiment, but we dont know how they performed during encoding so we discard them)
for worker_id in encoding_df['WorkerId']:
    if worker_id not in qualification_for_test_df.index:
        change_msg='- is not in the shelf'
        if worker_id in new_shelf_dict.keys():
            cur_values=new_shelf_dict[worker_id]
            new_values=cur_values.copy()
            new_values[0]=999
            new_shelf_dict[worker_id]=new_values
            change_msg=f'- shelf entry changed from: {cur_values} to {new_values}'

        print(f'{worker_id} did not have an encoding file ' + change_msg)

A1SJLXB0XFCJ97 did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]
ATUS3Q9I9EJNZ did not have an encoding file - shelf entry changed from: [48, None] to [999, None]
AW1EN7PL054OB did not have an encoding file - shelf entry changed from: [999, 999, 999] to [999, 999, 999]


In [19]:
#this section find participants that have a valid shelf entree but are not in the worker list (at all) 
#going over the worker list and not the results list because it is possible these participant were entered into the shelf due to a previous batchwoekre
for worker_id in new_shelf_dict.keys():
    cur_values=new_shelf_dict[worker_id]
    if cur_values[0]<=max_csv_number: #if its a participant with a valid csv number
        if worker_id not in workers_df['Worker ID'].values:
            new_values=cur_values.copy()
            new_values[0]=999
            change_msg=f' - shelf entry changed from: {cur_values} to {new_values}'
            new_shelf_dict[worker_id]=new_values
            print(f'{worker_id} absent from WORKERS LIST ' + change_msg)


A1SJLXB0XFCJ92 absent from WORKERS LIST  - shelf entry changed from: [13, 1673368956444] to [999, 1673368956444]
A2R5ZHND3OQJAV absent from WORKERS LIST  - shelf entry changed from: [23, 1673372772271] to [999, 1673372772271]


In [20]:
#this section extracts the names of the remaining valid participants in the shelf: 
valid_participants_in_shelf_for_test=[]
valid_participants_csvnumber_in_shelf_for_test=[]
for worker_id in new_shelf_dict.keys():
    cur_values=new_shelf_dict[worker_id]
    if cur_values[0]<=max_csv_number: #if its a participant with a valid csv number
        if worker_id in workers_df['Worker ID'].values:
            valid_participants_in_shelf_for_test.append(worker_id)
            valid_participants_csvnumber_in_shelf_for_test.append(cur_values[0])

valid_shelf_participants_info_df=pd.DataFrame({'Worker ID':valid_participants_in_shelf_for_test,'numeric csv':valid_participants_csvnumber_in_shelf_for_test},index=valid_participants_in_shelf_for_test)
valid_shelf_participants_info_df

,Worker ID,numeric csv
A21UA6O7ZFAIQJ,A21UA6O7ZFAIQJ,33
ABVBYS63BZWZ6,ABVBYS63BZWZ6,54
AN4P0SU1YSPF2,AN4P0SU1YSPF2,30
AOOLS8280CL0Z,AOOLS8280CL0Z,7
A2I6ZALE49CVSC,A2I6ZALE49CVSC,0
A1TPGXT718D68E,A1TPGXT718D68E,1
A2MOU3YD664C0H,A2MOU3YD664C0H,2
A2LA3NEWBU30QT,A2LA3NEWBU30QT,3
A3DP9IH8W5OXJL,A3DP9IH8W5OXJL,4
AMAEYJH3DWA1V,AMAEYJH3DWA1V,5


In [21]:
valid_shelf_participants_info_df.join(qualification_for_test_df)

,Worker ID,numeric csv,workerID,arrow_acc,mean_arrow_RT,in_encoding_workers_list
A21UA6O7ZFAIQJ,A21UA6O7ZFAIQJ,33,NaN,NaN,NaN,NaN
ABVBYS63BZWZ6,ABVBYS63BZWZ6,54,NaN,NaN,NaN,NaN
AN4P0SU1YSPF2,AN4P0SU1YSPF2,30,NaN,NaN,NaN,NaN
AOOLS8280CL0Z,AOOLS8280CL0Z,7,NaN,NaN,NaN,NaN
A2I6ZALE49CVSC,A2I6ZALE49CVSC,0,A2I6ZALE49CVSC,1.0,0.605320,1.0
A1TPGXT718D68E,A1TPGXT718D68E,1,A1TPGXT718D68E,1.0,1.272320,1.0
A2MOU3YD664C0H,A2MOU3YD664C0H,2,A2MOU3YD664C0H,1.0,1.046200,1.0
A2LA3NEWBU30QT,A2LA3NEWBU30QT,3,A2LA3NEWBU30QT,1.0,1.388880,1.0
A3DP9IH8W5OXJL,A3DP9IH8W5OXJL,4,A3DP9IH8W5OXJL,1.0,0.766260,1.0
AMAEYJH3DWA1V,AMAEYJH3DWA1V,5,AMAEYJH3DWA1V,1.0,1.277000,1.0


In [22]:
#this is the updated shelf: <<<<<< copy >>>>> from here and place in the shelf location in pavlovia.org
import json
json.dumps(new_shelf_dict)

'{"A21UA6O7ZFAIQJ": [33, 1672618541424, 1673265610931], "ABVBYS63BZWZ6": [54, 1672691441654, 1673184321835, 1673268961032], "AN4P0SU1YSPF2": [30, 1672608091759, 1673108004931, 1673115273107, 1673136507681, 1673150501613, 1673188398470], "AOOLS8280CL0Z": [7, 1672601000146, 1673111749669, 1673137735525, 1673185641801], "A2I6ZALE49CVSC": [0, 1673367367444], "A1TPGXT718D68E": [1, 1673367560992], "A2MOU3YD664C0H": [2, 1673367619383], "A2LA3NEWBU30QT": [3, 1673367650470], "A3DP9IH8W5OXJL": [4, 1673367667732], "AMAEYJH3DWA1V": [5, 1673367724382], "AYCD7UMZHW4WD": [999, 1673367821150], "ATH176GTV3ABP": [8, 1673368064110], "A1M1W56A3UR5PE": [9, 1673368158334], "A3S3WYVCVWW8IZ": [10, 1673368196632], "A21ZYNCWJLHMCY": [999, 999, 999], "A1SJLXB0XFCJ97": [999, 999, 999], "A2RSBH1E3TEESG": [11, 1673368592968], "A3QO3HIV92H659": [12, 1673368687949], "A1SJLXB0XFCJ92": [999, 1673368956444], "A2T5CROJ0FWIJY": [14, 1673369034005], "AV55CMG8KR1YH": [999, 999, 999], "A2OT9ZJC4MQBS8": [16, 1673368812171], "